In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(r'./NLP_Course_1_Dataset.csv')

In [3]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [46]:
# 1 - lowercase
# 2 - supprimer les !, ?, ; ... ) ( " 
# 3- lemmatization / stemming 
# supprimer les  <br/>
# supprimer les urls
# stopwords (and, this, are....)
# i'll => i will  ...
# {"don't":"do not"}

from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

def clean_review(text):
    """
    Clean the input text 
    """
    cleaned_text = text.lower()
    #cleaned_text =  re.sub(r'^https?:\/\/.*[\r\n]*', '', cleaned_text)
    cleaned_text = re.sub('<.*>', '', cleaned_text)
    cleaned_text = re.sub('[^a-zA-Z0-9\s]', '', cleaned_text)
    cleaned_text = cleaned_text.split()
    cleaned_text = [word for word in cleaned_text if word not in stopwords and len(word) > 1]
    
    return " ".join(cleaned_text)

clean_review(df.review[907])

'following directly story left part one second half sets telling inevitable downfall much grim side mans legacy exactly direct contrast first feature part two represents shift che pride glory revolutionised country chestruggling liberator country previous ties change setting isnt aesthetic autumn spring greys woodlands comes change tone heart feature replacing optimism predecessor cynical battered bruised reality aligned new struggle yet che would go say himselfsuch struggle best told exactly thata struggle part one certainly helped document initial surge power revolutionary guerrilla acquired part two takes much refined callous bleak segment ches life ambition gives assertive portrayal poignant tragic tangible easy grasp manner review jamie robert ward httpwwwinvocusnet'

In [47]:
df['cleaned_review'] = df['review'].apply(lambda x: clean_review(x))

In [48]:
df

,review,sentiment,cleaned_review
0,One of the other reviewers has mentioned that ...,positive,one reviewers mentioned watching oz episode yo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production realism really com...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...
...,...,...,...
49995,I thought this movie did a down right good job...,positive,thought movie right good job wasnt creative or...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,I am a Catholic taught in parochial elementary...,negative,catholic taught parochial elementary schools n...
49998,I'm going to have to disagree with the previou...,negative,im going disagree previous comment side maltin...


In [ ]:
# 1- Construire les vecteurs BoW et Tfidf (import sklearn / CountVectorizer & TfidfVectorizer)

In [58]:
from sklearn.model_selection import train_test_split
X = df['cleaned_review'].values.tolist() 
y = df['sentiment'].values.tolist()


In [59]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer

In [60]:
cv = CountVectorizer(stop_words='english')
cv_X = cv.fit_transform(X)
#cv_test = cv.fit_transform(X_test)


In [84]:
cv_X

<50000x142072 sparse matrix of type '<class 'numpy.int64'>'
	with 2692668 stored elements in Compressed Sparse Row format>

In [88]:
tf = TfidfVectorizer(stop_words='english')
tf_X = tf.fit_transform(X)
#tf_test = tf.fit_transform(X_test)

In [89]:
print(tf_train.shape)

(40000, 123942)


In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [65]:
print(cv_X.shape)
print(tf_X.shape)

(50000, 142072)
(50000, 142072)


In [68]:
# Bag of Words
cv_train = cv.transform(X_train)
cv_test = cv.transform(X_test)

In [69]:
from sklearn import preprocessing

In [70]:
le = preprocessing.LabelEncoder()

In [71]:
le.fit(df.sentiment.unique())

LabelEncoder()

In [73]:
le.classes_

array(['negative', 'positive'], dtype=object)

In [75]:
train_sentiments = le.fit_transform(y_train)
test_sentiments = le.fit_transform(y_test)

In [78]:
train_sentiments
test_sentiments

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [80]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
bow_lr = lr.fit(cv_train, train_sentiments)
print(bow_lr)

LogisticRegression()


C:\Users\hazourah\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [82]:
bow_test_predictions = bow_lr.predict(cv_test)

In [83]:
print(bow_test_predictions)
print(test_sentiments)

[1 1 0 ... 1 1 1]
[1 1 0 ... 1 0 1]


In [85]:
c = 0
for x,y in zip(test_sentiments, bow_test_predictions):
    if x == y:
        c+=1
    
print('Accuracy:',c/len(test_sentiments))

Accuracy: 0.8467


In [90]:
from sklearn.linear_model import LogisticRegression

tf_train = tf.transform(X_train)
tf_test = tf.transform(X_test)

lr = LogisticRegression()
tfidf_lr = lr.fit(tf_train, train_sentiments)
print(tfidf_lr)

LogisticRegression()


In [93]:
# samir.rabiai@universite-paris-saclay.fr
tfidf_test_predictions = tfidf_lr.predict(tf_test)

In [94]:
c = 0
for x,y in zip(test_sentiments, tfidf_test_predictions):
    if x == y:
        c+=1
    
print('Accuracy:',c/len(test_sentiments))

Accuracy: 0.8591


In [4]:
df.sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [6]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [7]:
df.groupby('sentiment').count()

,review
sentiment,
negative,25000
positive,25000


In [14]:
df[df['review'].str.contains('http')].loc[907]['review']

"Following directly from where the story left off in part one, the second half which sets about telling the inevitable downfall and much more grim side of the man's legacy is exactly as such. In direct contrast to the first feature, part two represents a shift from Che the pride and glory of a revolutionised country, to Che\x97struggling liberator of a country to which he has no previous ties. The change of setting isn't just aesthetic; from the autumn and spring greys of the woodlands comes a change of tone and heart to the feature, replacing the optimism of the predecessor with a cynical, battered and bruised reality aligned to an all new struggle. Yet, as Che would go on to say himself\x97such a struggle is best told exactly as that\x97a struggle. While Part One certainly helped document that initial surge to power that the revolutionary guerrilla acquired through just that, Part Two takes a much more refined, callous and bleak segment of Che's life and ambition, and gives it an ass

In [15]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [97]:
df[df.review.str.contains('horror')]['review'].iloc[0]

"I saw this movie when I was about 12 when it came out. I recall the scariest scene was the big bird eating men dangling helplessly from parachutes right out of the air. The horror. The horror.<br /><br />As a young kid going to these cheesy B films on Saturday afternoons, I still was tired of the formula for these monster type movies that usually included the hero, a beautiful woman who might be the daughter of a professor and a happy resolution when the monster died in the end. I didn't care much for the romantic angle as a 12 year old and the predictable plots. I love them now for the unintentional humor.<br /><br />But, about a year or so later, I saw Psycho when it came out and I loved that the star, Janet Leigh, was bumped off early in the film. I sat up and took notice at that point. Since screenwriters are making up the story, make it up to be as scary as possible and not from a well-worn formula. There are no rules."

In [99]:
df[df.review.str.contains('police')]['review'].iloc[0]

'"Ardh Satya" is one of the finest film ever made in Indian Cinema. Directed by the great director Govind Nihalani, this one is the most successful Hard Hitting Parallel Cinema which also turned out to be a Commercial Success. Even today, Ardh Satya is an inspiration for all leading directors of India.<br /><br />The film tells the Real-life Scenario of Mumbai Police of the 70s. Unlike any Police of other cities in India, Mumbai Police encompasses a Different system altogether. Govind Nihalani creates a very practical Outlay with real life approach of Mumbai Police Environment.<br /><br />Amongst various Police officers & colleagues, the film describes the story of Anand Velankar, a young hot-blooded Cop coming from a poor family. His father is a harsh Police Constable. Anand himself suffers from his father\'s ideologies & incidences of his father\'s Atrocities on his mother. Anand\'s approach towards immediate action against crime, is an inert craving for his own Job satisfaction. The

In [100]:
# Word2Vec 

In [102]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [103]:
from nltk import sent_tokenize

In [104]:
from gensim.utils import simple_preprocess

In [105]:
import gensim

In [140]:
corpus = [sent_tokenize(review) for review in df.cleaned_review.values.tolist()]

In [141]:
print(corpus[0])

['one reviewers mentioned watching oz episode youll hooked right exactly happened mei would say main appeal show due fact goes shows wouldnt dare forget pretty pictures painted mainstream audiences forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards wholl sold nickel inmates wholl kill order get away well mannered middle class inmates turned prison bitches due lack street skills prison experience watching oz may become comfortable uncomfortable viewingthats get touch darker side']


In [157]:
words = []
labels = []
for rev,label in zip(df.cleaned_review.values.tolist(), df.sentiment.values.tolist()):
    sent_rev = sent_tokenize(rev)
    for word in sent_rev:
        words.append(simple_preprocess(word))
        labels.append(label)
model_w2v = gensim.models.Word2Vec(words, window = 5, min_count = 1)

In [160]:
model_w2v.corpus_count
len(words)
len(labels)

49972

In [240]:
def review_embeddings(review,model_w2v, method:str = 'avg'):
    """ 
    Return the Text vector using the average or sum of word embeddings given by Word2Vec
    """
    if method == 'avg':
        return np.mean([model_w2v.wv[word] for word in review if word in model_w2v.wv.index_to_key], axis = 0)
    return np.sum([model_w2v.wv[word] for word in review if word in model_w2v.wv.index_to_key], axis = 0)

In [ ]:
X_reviews = [review_embeddings(review,wv) for review in words]
X_reviews = np.array(X_reviews)
label_dict  ={'positive': 1, 'negative':0}
y_reviews = [label_dict[label] for label in labels]
#X_reviews = np.array(X_reviews)
#X_reviews = np.reshape(X_reviews,(X_reviews.shape[0],100))
y_reviews = y_reviews
X_train, X_test, y_train, y_test = train_test_split(X_reviews, y_reviews, test_size =0.2)

C:\Users\hazourah\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\hazourah\Anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
X_reviews.shape

In [ ]:
#model_w2v.train(corpus, total_examples=1, epochs=5)
model_w2v.wv.most_similar('one')

In [ ]:
lr_w2v = LogisticRegression()

In [ ]:
X_train_corr = []
y_train_corr = []
for i,y in zip(X_train,y_train):
    try:
        if i.shape[0] == 100:
            X_train_corr.append(i)
            y_train_corr.append(y)
    except:
        print(i)

In [ ]:
X_test_corr = []
y_test_corr = []
for i,y in zip(X_test,y_test):
    try:
        if i.shape[0] == 100:
            X_test_corr.append(i)
            y_test_corr.append(y)
    except:
        print(i)

In [ ]:
X_train_corr = np.array(X_train_corr)

In [ ]:
X_test_corr = np.array(X_test_corr)

In [ ]:
lr_w2v.fit(X_train_corr, y_train_corr)

In [ ]:
lr_w2v.predict(X_test_corr)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test_corr,lr_w2v.predict(X_test_corr))

In [243]:
def review_embeddings(review,model_w2v, method:str = 'avg'):
    """ 
    Return the Text vector using the average or sum of word embeddings given by Word2Vec
    """
    if method == 'avg':
        return np.mean([model_w2v[word] for word in review if word in model_w2v], axis = 0)
    return np.sum([model_w2v.wv[word] for word in review if word in model_w2v], axis = 0)